# Expected Value of Control - OptimizationControlMechanism (EVC - OCM)

*Installation and Setup*

In [ ]:
%%capture
%pip install psyneulink

In [ ]:
import psyneulink as pnl

First, we model the decision-making process using a Drift Diffusion Model (DDM) with an analytical solution. To model adaptive control, we use reward as input. The reward input can be used to optimize threshold and drift-rate of the DDM.

We can think of this as finding the balance between the cost of control, the time it takes to make a decision, and the accuracy of the decision.

In [ ]:
stimulus =  pnl.ProcessingMechanism(name='Stimulus') # Input Stimulus can be seen as SignalToNoise Ratio (SNR) the higher the easier the decision
reward = pnl.ProcessingMechanism(name='Reward') # Reward Mechanism

decision = pnl.DDM(
    name='Decision',
    function=pnl.DriftDiffusionAnalytical(
        drift_rate=1.0,
        threshold=1.0,
        noise=0.5,
        starting_value=0,
        non_decision_time=0.45
    ),
    output_ports=[pnl.DECISION_VARIABLE,
                  pnl.RESPONSE_TIME,
                  pnl.PROBABILITY_UPPER_THRESHOLD], # < -- Here, we add the output ports that we want to monitor for optimization of control
)

Let's create the composition (first without the control):

In [ ]:
comp = pnl.Composition(name="EVC-OCM")
comp.add_node(reward)
comp.add_linear_processing_pathway([stimulus, decision])

comp.show_graph(output_fmt='jupyter')

Now, we add a controller.

First, we create the objective mechanism that will specify the optimization goal. In this case, we want to optimize the reward and probability of being correct, as well as balancing the response time.

In [ ]:
objective_mechanism = pnl.ObjectiveMechanism(
    name='OCM Objective Mechanism',
    function=pnl.LinearCombination(operation=pnl.PRODUCT),
    monitor=[
        reward,  # <-- When a mechanism is monitored, its default output port is monitored
        decision.output_ports[pnl.PROBABILITY_UPPER_THRESHOLD], # <-- We monitor the PROBABILITY_UPPER_THRESHOLD output port previously specified as the decision mechanism
        (decision.output_ports[pnl.RESPONSE_TIME], -1, 1)] # <-- In addition to an output port, we can also specify a transformation (weight and exponent),here we
)

Now, we can create the controller and add it to the composition:

In [ ]:
controller = pnl.OptimizationControlMechanism(
    name = 'OCM',
    agent_rep=comp,
    state_features=[stimulus.input_port, reward.input_port],
    state_feature_function=pnl.AdaptiveIntegrator(rate=0.5), # <- Instead of the current input the input and the last state of the input are integrated via this function (This simulates a "sliding window over average" of input and reward)
    objective_mechanism=objective_mechanism,
    function=pnl.GridSearch(),
    control_signals=[
        pnl.ControlSignal(modulates=(pnl.DRIFT_RATE, decision), allocation_samples=[0.1, 0.3, 0.5, 0.7, 0.9]),
        pnl.ControlSignal(modulates=(pnl.THRESHOLD, decision), allocation_samples=[0.1, 0.3, 0.5, 0.7, 0.9]),
    ]
)

comp.add_controller(controller)

Let's look at the composition again, this time with the controller:

In [ ]:
comp.show_graph(show_controller=True)

And run this on some example data:

In [ ]:
stim_list_dict = {
    stimulus: [0.5, 0.123],
    reward: [10, 20]
}

comp.run(inputs=stim_list_dict)

print(comp.results)